# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [33]:
import numpy as np 
import pandas as pd 
import math 
import requests
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [48]:
#grabbing list of S&P 500 Companies 
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]

symbols_list = (df['Symbol'])
sectors_list = df['GICS Sector']


stocks = symbols_list.to_frame()
sectors = sectors_list.to_frame()

IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


    Symbol
0      MMM
1      AOS
2      ABT
3     ABBV
4     ABMD
..     ...
500    YUM
501   ZBRA
502    ZBH
503   ZION
504    ZTS

[505 rows x 1 columns]


In [ ]:
#anirudh was here dfdfjdfjdjf

In [ ]:
# can you see this?

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [35]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [37]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []




my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [38]:

simple_list = []

for symbol_string in symbol_strings: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol == 'WLTW':
            pass
        else:
            simple_list.append([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['peRatio'], 'N/A'])



final_dataframe = pd.DataFrame(simple_list, columns=my_columns)






        

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [39]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [40]:
# my_columns = ['Symbol', 'Price', 'Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio' ,'Number of Shares to Buy']
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']

ev_to_ebitda = enterprise_value/ebitda
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [41]:
rv_columns = [
    'Symbol',
    'Price',
    'Number of Shares to Buy',
    'PE Ratio',
    'PE Percentile',
    'PB Ratio',
    'PB Percentile',
    'PS Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'PEG Ratio',
    'DE',
    'RV Score'
]


In [42]:
simple_list = []

for symbol_string in symbol_strings: 
    batch_api_call_url = batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        # if symbol == 'WLTW':
        #     pass
        # else:
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        
        except TypeError:
            ev_to_ebitda = np.NaN

        ebitda = data[symbol]['advanced-stats']['EBITDA']
        
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        simple_list.append([symbol, 
                            data[symbol]['quote']['latestPrice'], 
                            'N/A', 
                            data[symbol]['quote']['peRatio'],
                            'N/A', 
                            data[symbol]['advanced-stats']['priceToBook'],
                            'N/A',
                            data[symbol]['advanced-stats']['priceToSales'],
                            'N/A',
                            ev_to_ebitda,
                            'N/A',
                            ev_to_gross_profit,
                            'N/A',
                            data[symbol]['advanced-stats']['pegRatio'],
                            data[symbol]['advanced-stats']['debtToEquity'],
                            'N/A'])


rv_dataframe = pd.DataFrame(simple_list, columns = rv_columns)

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [43]:
rv_dataframe

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,PEG Ratio,DE,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [ ]:
# rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [44]:

exclude = []

# print(rv_columns[3:12:2])


# print(value)
for column in rv_columns[3:12:2]:

    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace= True)



## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [45]:
metrics = {}

for column in rv_dataframe.columns[3:12:2]:
    column1 = rv_dataframe.columns.get_loc(column)
    metrics.update({column : rv_dataframe.columns[column1 + 1] })

for metric in metrics: 
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])



## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [46]:
from statistics import mean



for row in rv_dataframe.index:
    pe = rv_dataframe.loc[row, 'PE Percentile']
    pb = rv_dataframe.loc[row, 'PB Percentile']
    ps = rv_dataframe.loc[row, 'PS Percentile']
    ev_ebitda = rv_dataframe.loc[row, 'EV/EBITDA Percentile']
    ev_gp = rv_dataframe.loc[row, 'EV/GP Percentile'] 
    rv_dataframe.loc[row, 'RV Score'] = mean([pe, pb, ps, ev_ebitda, ev_gp])

print(rv_dataframe)


Empty DataFrame
Columns: [Symbol, Price, Number of Shares to Buy, PE Ratio, PE Percentile, PB Ratio, PB Percentile, PS Ratio, PS Percentile, EV/EBITDA, EV/EBITDA Percentile, EV/GP, EV/GP Percentile, PEG Ratio, DE, RV Score]
Index: []


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [47]:
best_value_stocks = rv_dataframe.sort_values(by = 'RV Score', ascending = True)
best_value_stocks = best_value_stocks[:50]
best_value_stocks.reset_index(drop = True, inplace = True )

best_value_stocks

,Symbol,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,PEG Ratio,DE,RV Score


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()


In [ ]:
for row in best_value_stocks.index:
    best_value_stocks.loc[row, 'Number of Shares to Buy'] = math.floor(float(portfolio_size)/best_value_stocks.loc[row, 'Price'])

best_value_stocks

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: